In [1]:
cd ../src

/home/alberto/Work/incomplete_multiview_clustering/src


In [2]:
from mvlearn.datasets import load_nutrimouse
import pandas as pd
from utils import DatasetUtils
from algorithms import Concat

In [3]:
dict_data = load_nutrimouse()
gene = pd.DataFrame(dict_data['gene'], columns = dict_data['gene_feature_names'])
lipid = pd.DataFrame(dict_data['lipid'], columns = dict_data['lipid_feature_names'])
mvd = [gene, lipid]
sample_views = DatasetUtils.add_sample_views(mvd = mvd, p = [0.2, 0.5])
imvd = DatasetUtils.add_missing_views(mvd = mvd, sample_views = sample_views)
for data in imvd:
    print(data.shape)
    display(data.head())

(37, 120)


,X36b4,ACAT1,ACAT2,ACBP,ACC1,ACC2,ACOTH,ADISP,ADSS1,ALDH3,...,cHMGCoAS,cMOAT,eif2g,hABC1,iBABP,iBAT,iFABP,iNOS,mABC1,mHMGCoAS
0,-0.42,-0.65,-0.84,-0.34,-1.29,-1.13,-0.93,-0.98,-1.19,-0.68,...,-1.15,-0.78,-1.23,-1.16,-0.78,-1.65,-1.14,-1.24,-0.85,-0.03
1,-0.44,-0.68,-0.91,-0.32,-1.23,-1.06,-0.99,-0.97,-1.00,-0.62,...,-0.95,-0.73,-1.02,-1.11,-0.73,-1.67,-1.03,-1.20,-0.84,-0.12
2,-0.48,-0.74,-1.10,-0.46,-1.30,-1.09,-1.06,-1.08,-1.18,-0.75,...,-0.93,-0.89,-1.14,-1.25,-0.89,-1.89,-1.16,-1.35,-0.96,-0.12
3,-0.45,-0.69,-0.65,-0.41,-1.26,-1.09,-0.93,-1.02,-1.07,-0.71,...,-1.10,-0.93,-1.10,-1.17,-0.82,-1.70,-1.17,-1.25,-0.87,-0.12
4,-0.42,-0.71,-0.54,-0.38,-1.21,-0.89,-1.00,-0.95,-1.08,-0.76,...,-0.94,-0.84,-1.10,-1.14,-0.71,-1.68,-1.15,-1.20,-0.82,-0.29


(21, 21)


,C140,C160,C180,C161n9,C161n7,C181n9,C181n7,C201n9,C203n9,C182n6,...,C202n6,C203n6,C204n6,C224n6,C225n6,C183n3,C203n3,C205n3,C225n3,C226n3
0,0.34,26.45,10.22,0.35,3.10,16.98,2.41,0.26,0.00,8.93,...,0.00,0.78,3.07,0.00,0.00,5.97,0.37,8.62,1.75,10.39
2,0.36,23.70,8.96,0.55,2.65,22.89,3.96,0.26,0.19,16.06,...,0.33,1.51,13.27,0.54,1.77,0.00,0.00,0.00,0.22,2.51
3,0.22,25.48,8.14,0.49,2.82,21.92,2.52,0.00,0.00,13.89,...,0.00,1.10,3.92,0.00,0.00,0.49,0.00,2.99,1.04,14.99
4,0.37,24.80,9.63,0.46,2.85,21.38,2.96,0.30,0.27,14.55,...,0.23,1.58,11.85,0.32,0.44,0.42,0.00,0.30,0.35,6.69
7,0.34,28.63,9.95,0.29,3.27,17.02,1.99,0.31,0.00,7.75,...,0.00,0.72,2.64,0.00,0.00,6.01,0.39,7.96,2.33,10.40


In [4]:
from utils import ConcatenateViews

ConcatenateViews().fit_transform(imvd)

,X36b4,ACAT1,ACAT2,ACBP,ACC1,ACC2,ACOTH,ADISP,ADSS1,ALDH3,...,C202n6,C203n6,C204n6,C224n6,C225n6,C183n3,C203n3,C205n3,C225n3,C226n3
0,-0.42,-0.65,-0.84,-0.34,-1.29,-1.13,-0.93,-0.98,-1.19,-0.68,...,0.00,0.78,3.07,0.00,0.00,5.97,0.37,8.62,1.75,10.39
1,-0.44,-0.68,-0.91,-0.32,-1.23,-1.06,-0.99,-0.97,-1.00,-0.62,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,-0.48,-0.74,-1.10,-0.46,-1.30,-1.09,-1.06,-1.08,-1.18,-0.75,...,0.33,1.51,13.27,0.54,1.77,0.00,0.00,0.00,0.22,2.51
3,-0.45,-0.69,-0.65,-0.41,-1.26,-1.09,-0.93,-1.02,-1.07,-0.71,...,0.00,1.10,3.92,0.00,0.00,0.49,0.00,2.99,1.04,14.99
4,-0.42,-0.71,-0.54,-0.38,-1.21,-0.89,-1.00,-0.95,-1.08,-0.76,...,0.23,1.58,11.85,0.32,0.44,0.42,0.00,0.30,0.35,6.69
5,-0.43,-0.69,-0.80,-0.32,-1.13,-0.79,-0.93,-0.97,-1.07,-0.75,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,-0.53,-0.62,-1.00,-0.44,-1.22,-1.00,-0.94,-0.94,-1.05,-0.67,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,-0.49,-0.69,-0.91,-0.37,-1.29,-1.06,-1.05,-1.02,-1.16,-0.75,...,0.00,0.72,2.64,0.00,0.00,6.01,0.39,7.96,2.33,10.40
8,-0.36,-0.66,-0.74,-0.39,-1.15,-1.08,-0.88,-0.98,-1.05,-0.66,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,-0.40,-0.60,-0.55,-0.25,-1.22,-1.13,-0.81,-0.99,-1.10,-0.77,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
from mvlearn.compose import ConcatMerger

ConcatenateViews().fit_transform(imvd)

,X36b4,ACAT1,ACAT2,ACBP,ACC1,ACC2,ACOTH,ADISP,ADSS1,ALDH3,...,C202n6,C203n6,C204n6,C224n6,C225n6,C183n3,C203n3,C205n3,C225n3,C226n3
0,-0.42,-0.65,-0.84,-0.34,-1.29,-1.13,-0.93,-0.98,-1.19,-0.68,...,0.00,0.78,3.07,0.00,0.00,5.97,0.37,8.62,1.75,10.39
1,-0.44,-0.68,-0.91,-0.32,-1.23,-1.06,-0.99,-0.97,-1.00,-0.62,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,-0.48,-0.74,-1.10,-0.46,-1.30,-1.09,-1.06,-1.08,-1.18,-0.75,...,0.33,1.51,13.27,0.54,1.77,0.00,0.00,0.00,0.22,2.51
3,-0.45,-0.69,-0.65,-0.41,-1.26,-1.09,-0.93,-1.02,-1.07,-0.71,...,0.00,1.10,3.92,0.00,0.00,0.49,0.00,2.99,1.04,14.99
4,-0.42,-0.71,-0.54,-0.38,-1.21,-0.89,-1.00,-0.95,-1.08,-0.76,...,0.23,1.58,11.85,0.32,0.44,0.42,0.00,0.30,0.35,6.69
5,-0.43,-0.69,-0.80,-0.32,-1.13,-0.79,-0.93,-0.97,-1.07,-0.75,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,-0.53,-0.62,-1.00,-0.44,-1.22,-1.00,-0.94,-0.94,-1.05,-0.67,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,-0.49,-0.69,-0.91,-0.37,-1.29,-1.06,-1.05,-1.02,-1.16,-0.75,...,0.00,0.72,2.64,0.00,0.00,6.01,0.39,7.96,2.33,10.40
8,-0.36,-0.66,-0.74,-0.39,-1.15,-1.08,-0.88,-0.98,-1.05,-0.66,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,-0.40,-0.60,-0.55,-0.25,-1.22,-1.13,-0.81,-0.99,-1.10,-0.77,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


We create a random multi-view dataset with 10 samples.

Now we will remove some views from the samples.

In [32]:
from utils import get_sample_views
from sumo.modes.prepare.similarity import feature_to_adjacency
from sumo.network import MultiplexNet
from sumo.modes.run.solvers.unsupervised_sumo import UnsupervisedSumoNMF
from sumo.modes.run.run import run_thread_wrapper


class SUMO():
    
    def __init__(self, k: int, method = ['euclidean'], missing : list = [0.1], neighbours : float = 0.1, alpha: float = 0.5, sparsity : list = [0.1], repetitions : int = 60,
                 cluster_method : str = "max_value", max_iter : int =500, tol : float = 1e-5, subsample : float = 0.05, calc_cost : int = 20,
                 h_init :int = None, n_jobs : int = 1, rep : int = 5, random_state : int = None):
        self.method = method
        self.missing = missing
        self.neighbours = neighbours
        self.alpha = alpha
        self.k = k
        self.sparsity = sparsity
        self.repetitions = repetitions
        self.cluster_method = cluster_method
        self.max_iter = max_iter
        self.tol = tol
        self.subsample = subsample
        self.calc_cost = calc_cost
        self.h_init = h_init
        self.n_jobs = n_jobs
        self.rep = rep
        self.random_state = random_state
        
        self.graph_ = None
        self.nmf_ = None
        self.priors_ = None
        
        if self.repetitions < 1:
            raise ValueError("Incorrect value of 'repetitions' parameter")
        if self.n_jobs < 1:
            raise ValueError("Incorrect number of threads")
        if self.subsample > 0.5 or self.subsample < 0:
            # do not allow for removal of more then 50% of samples in each run
            raise ValueError("Incorrect value of 'subsample' parameter")
        if self.rep < 1:
            # number of times additional consensus matrix will be created
            raise ValueError("Incorrect value of 'rep' parameter")
        if self.random_state is not None and self.random_state < 0:
            raise ValueError("Seed value cannot be negative")
        self.runs_per_con = max(round(self.repetitions * 0.8), 1)  # number of runs per consensus matrix creation
        
        if len(self.k) > 2 or (len(self.k) == 2 and self.k[0] > self.k[1]):
            raise ValueError("Incorrect range of k values")
        elif len(self.k) == 2:
            self.k = list(range(self.k[0], self.k[1] + 1))        
        
        
        
    def fit(self, X, y = None):
        if len(self.missing) == 1:
            self.missing = [self.missing[0]] * len(X)
        if len(self.method) == 1:
            self.method = [self.method[0]] * len(X)
        elif len(X) != len(self.method):
            raise ValueError("Number of matrices extracted from input files and number of similarity methods does not correspond")

        all_samples = get_sample_views(imvd = X).index.values
        self.similarity_ = {}
        adj_matrices = []
        # create adjacency matrices
        for view_idx, view_data in enumerate(X):
            # create adjacency matrix
            a = feature_to_adjacency(view_data.values, missing=self.missing[view_idx], method=self.method[view_idx], n=self.neighbours, alpha=self.alpha)
            # add matrices to output arrays
            adj_matrices.append(a)
            self.similarity_[str(view_idx)] = a
            self.similarity_[f"f{view_idx}"] = view_data
            
        ##################################################################
        if self.h_init is not None:
            if self.h_init >= len(adj_matrices) or self.h_init < 0:
                raise ValueError("Incorrect value of h_init")
                
        # create multilayer graph
        self.graph = MultiplexNet(adj_matrices=adj_matrices, node_labels= all_samples)
        n_sub_samples = round(all_samples.size * self.subsample)
        # create solver
        self.nmf = UnsupervisedSumoNMF(graph=self.graph, nbins=self.repetitions, bin_size=self.graph.nodes - n_sub_samples, rseed=self.seed)
        global _sumo_run
        _sumo_run = self  # this solves multiprocessing issue with pickling
        # run factorization for every (eta, k)
        cophenet_list = []
        pac_list = []
        for k in self.k:
            if self.n_jobs == 1:
                results = [_run_factorization(sparsity=sparsity, k=k, sumo_run=_sumo_run) for sparsity in self.sparsity]
                sparsity_order = self.sparsity
            else:
                pool = mp.Pool(self.t)
                results = []
                sparsity_order = []
                iproc = 1
                for res in pool.imap_unordered(run_thread_wrapper, zip(self.sparsity, [k] * len(self.sparsity))):
                    results.append(res[0])
                    sparsity_order.append(res[1])
                    iproc += 1

            # select best result
            best_result = sorted(results, reverse=True)[0]
            best_eta = None

            quality_output = []
            for (result, sparsity) in zip(results, sparsity_order):
                quality_output.append(np.array([sparsity, result[0]]))
                if result[1] == best_result[1]:
                    best_eta = sparsity

            # summarize results
            assert best_eta is not None
            return best_result
#             out_arrays = load_npz(best_result[1])

#             cophenet_list.append(out_arrays["cophenet"])
#             pac_list.append(out_arrays["pac"])

#             # create text file with cluster labels
#             clusters = out_arrays['clusters']
#             with open(os.path.join(self.outdir, "k{}".format(k), "clusters.tsv"), 'w') as cl_file:
#                 cl_file.write("sample\tlabel\n")
#                 for row_idx in range(clusters.shape[0]):
#                     cl_file.write("{}\t{}\n".format(clusters[row_idx, 0], clusters[row_idx, 1]))

#             # create symlink to the selected best result
#             summary_outfile = os.path.join(self.outdir, "k{}".format(k), "sumo_results.npz")
#             if os.path.lexists(summary_outfile):
#                 # overwriting symlink
#                 os.remove(summary_outfile)

#             workdir = os.getcwd()
#             os.chdir(os.path.dirname(best_result[1]))
#             os.symlink(os.path.basename(best_result[1]), os.path.basename(summary_outfile))
#             os.chdir(workdir)
#             assert os.getcwd() == workdir

#             self.logger.info("Results (k = {}) saved to {}".format(k, summary_outfile))

#             plot_heatmap_seaborn(out_arrays['consensus'], labels=np.arange(self.graph.nodes),
#                                  title="Consensus matrix (K = {})".format(k),
#                                  file_path=os.path.join(self.plot_dir, "consensus_k{}.png".format(k)))
#             # TODO: change sample order

#         if len(cophenet_list) > 1 and len(pac_list) > 1:
#             cophenet_plot_path = os.path.join(self.plot_dir, "cophenet.png")
#             plot_metric(x=self.k, y=cophenet_list, xlabel="K", ylabel="cophenetic correlation coefficient",
#                         title="Cluster stability for different K values", file_path=cophenet_plot_path, color="red")
#             self.logger.info("#Cophentic correlation coefficient plot for different K values has " +
#                              "been saved to {}".format(cophenet_plot_path))

#             pac_plot_path = os.path.join(self.plot_dir, "pac.png")
#             plot_metric(x=self.k, y=pac_list, xlabel="K", ylabel="PAC",
#                         title="Proportion of ambiguous clusterings for different K values", file_path=pac_plot_path,
#                         color="blue")
#             self.logger.info("#Proportion of ambiguous clusterings plot for different K values has " +
#                              "been saved to {}".format(pac_plot_path))
#         return self


In [36]:
sumo = SUMO(k = [3])
sumo.fit(X = mvd)

ValueError: Different number of nodes in layers

In [2]:
from sumo.modes.prepare.similarity import feature_to_adjacency

In [4]:
estimator = Concat()
estimator.fit(X = imvd)

Concat()

In [5]:
estimator.predict(X = imvd)

array([3, 1, 7, 1, 1, 6, 6, 7, 3, 7, 5, 3, 7, 1, 4, 6, 5, 7, 3, 6, 6, 6,
       3, 6, 6, 3, 3, 1, 7, 4, 3, 4, 0, 4, 7, 6, 3, 6, 7, 6, 0, 1, 3, 1,
       2, 2, 5, 5, 6, 3, 1, 6, 0, 0, 6, 3, 6, 3, 0, 7, 0, 3, 3, 6, 2, 6,
       1, 7, 3, 7, 6, 7, 6, 3, 4, 6, 5, 1, 1, 3, 3, 4, 3, 1, 0, 7, 7, 2,
       4, 6, 5, 7, 1, 3, 0, 4, 3, 0, 0, 4], dtype=int32)